In [7]:
import torch
import numpy as np
import ipdb

In [8]:
def gaussian_kernel(kernel_size, sigma):
    x = np.arange(0, kernel_size, 1, float)
    y = x[:, np.newaxis]  # just like transpose x and add a new axis
    x0 = y0 = kernel_size // 2
    gaussian = np.exp(- ((x - x0) ** 2 + (y - y0) ** 2) / (2 * sigma ** 2))
    return gaussian

In [9]:
def grouped(iterable, n):
    return zip(*[iter(iterable)]*n)

In [10]:
def keypoint_to_gaussian_heatmap(new_kp_array, output_size, radius=3):
    '''
        img: torch tensor (single channel)
        keypoint: (x, y) (single point)
    '''
    # calculate gaussian range
    c = new_kp_array.shape[0]
    h, w = output_size
#     x, y, v = keypoint
    heatmap = np.zeros([c, h, w])
    tl_point = new_kp_array[:,:2] - radius
    br_point = new_kp_array[:,:2] + radius
    # Invalid keypoint position
    zero_idx = np.unique(np.concatenate((np.where(tl_point[:,0] >= w)[0],
                                        np.where(tl_point[:,1] >= h)[0],
                                        np.where(br_point[:,0] < 0)[0],
                                        np.where(br_point[:,1] < 0)[0],
                                        np.where(new_kp_array[:,2] < 1)[0])))
    cal_idx = set(range(c)).difference(zero_idx)

    kernel_size = 2 * radius + 1
    sigma = kernel_size / 4
    gaussian = np.array(gaussian_kernel(kernel_size, sigma))
    # Gaussian range
    g_xmin = np.round(np.maximum(0, -tl_point[:,0])).astype(int)
    g_xmax = np.round(np.minimum(w, br_point[:,0]) - tl_point[:,0]).astype(int)
    g_ymin = np.round(np.maximum(0, -tl_point[:,1])).astype(int)
    g_ymax = np.round(np.minimum(h, br_point[:,1]) - tl_point[:,1]).astype(int)
    
    # Image range
    heatmap_xmin = np.round(np.maximum(0, tl_point[:,0])).astype(int)
    heatmap_xmax = np.round(np.minimum(w, br_point[:,0])).astype(int)
    heatmap_ymin = np.round(np.maximum(0, tl_point[:,1])).astype(int)
    heatmap_ymax = np.round(np.minimum(h, br_point[:,1])).astype(int)

    delta_y = np.minimum(g_ymax-g_ymin, heatmap_ymax-heatmap_ymin)
    delta_x = np.minimum(g_xmax-g_xmin, heatmap_xmax-heatmap_xmin)
    
    for i in cal_idx:
        heatmap[i,heatmap_ymin[i]:heatmap_ymin[i]+delta_y[i], heatmap_xmin[i]:heatmap_xmin[i]+delta_x[i]] \
                = gaussian[g_ymin[i]:g_ymin[i]+delta_y[i], g_xmin[i]:g_xmin[i]+delta_x[i]]
    return torch.as_tensor(heatmap, dtype=torch.float32)

In [11]:
def transform_keypoint_coordinate(kp_array, bbox, output_size):
    '''
        transform and get final keypoint coordinate in final heatmap

        keypoint: (x, y)
        bbox: [xmin, ymin, xmax, ymax]
        input_size: (input_h, input_w)
        output_size: (output_h, output_w)
    '''
    # relative coordinate to the top_left point
    bbox_array = np.array(bbox)
    relative_tl = kp_array - bbox_array[:2]
    # calculating padding value
    input_h, input_w = bbox[3] - bbox[1], bbox[2] - bbox[0]
    output_h, output_w = output_size
    h = max(input_h, input_w * output_h / output_w)
    w = h * output_w / output_h
    delta_w = w - input_w
    delta_h = h - input_h
    # fix relative coordinate after padding
    relatvie_tl = relative_tl + np.array([delta_w/2, delta_h/2])
    ratio = output_h / h
    final_coordinate = relatvie_tl * ratio
    return final_coordinate

In [12]:
keypoints=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 59, 489, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
kp_array = np.asarray(keypoints).reshape(-1,3)
output_size = (80, 64)
bbox = (0, 293, 120, 611)
new_kp_xy = transform_keypoint_coordinate(kp_array[:,:2], bbox, output_size)
new_kp_array = np.concatenate((new_kp_xy, kp_array[:,-1:]), axis=1)
heatmaps = keypoint_to_gaussian_heatmap(new_kp_array, output_size)
print(np.where(heatmaps.numpy()>0))

(array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]), array([46, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48,
       48, 49, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 50, 51, 51, 51, 51,
       51, 51]), array([29, 30, 31, 32, 33, 34, 29, 30, 31, 32, 33, 34, 29, 30, 31, 32, 33,
       34, 29, 30, 31, 32, 33, 34, 29, 30, 31, 32, 33, 34, 29, 30, 31, 32,
       33, 34]))
